In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import torch.functional as F
from io import open
from PIL import Image
import pathlib
import cv2
from torch.autograd import Variable
from torchvision.models import squeezenet1_1

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [5]:
#Dataloader
#Path for training and testing directory
train_path='train'
test_path='test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=10, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=10, shuffle=True
)

In [6]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['banana', 'bucket', 'elephant', 'keyboard', 'orange', 'penguin', 'pizza', 'schoolbus', 'tea', 'tower']


In [8]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (10,3,64,64)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (10,12,64,64) #Shape= (10,12,100,100)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (10,12,64,64) #Shape= (10,12,100,100)
        self.lrelu1=nn.LeakyReLU()
        #Shape= (10,12,64,64) #Shape= (10,12,100,100)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (10,12,32,32) #Shape= (10,12,50,50)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (10,20,32,32) #Shape= (10,20,50,50)
        self.lrelu2=nn.LeakyReLU()
       #Shape= (10,20,32,32) #Shape= (10,20,50,50)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (10,32,32,32) #Shape= (10,32,50,50)
        self.bn3=nn.BatchNorm2d(num_features=32)
         #Shape= (10,32,32,32) #Shape= (10,32,50,50)
        self.lrelu3=nn.LeakyReLU()
         #Shape= (10,32,32,32) #Shape= (10,32,50,50)
        
        
        self.fc=nn.Linear(in_features=75*75*32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.lrelu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.lrelu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.lrelu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,75*75*32)
            
            
        output=self.fc(output)
            
        return output

In [9]:
model=ConvNet(num_classes=10).to(device)

In [10]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.00001)
loss_function=nn.CrossEntropyLoss()

In [11]:
num_epochs=5

In [12]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.JPEG'))
test_count=len(glob.glob(test_path+'/**/*.JPEG'))

In [13]:
print(train_count,test_count)

500 100


In [14]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint3.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(3.3126) Train Accuracy: 0.384 Test Accuracy: 0.03
Epoch: 1 Train Loss: tensor(1.2982) Train Accuracy: 0.678 Test Accuracy: 0.03
Epoch: 2 Train Loss: tensor(0.8622) Train Accuracy: 0.802 Test Accuracy: 0.03
Epoch: 3 Train Loss: tensor(0.4233) Train Accuracy: 0.86 Test Accuracy: 0.03
Epoch: 4 Train Loss: tensor(0.2488) Train Accuracy: 0.914 Test Accuracy: 0.05


In [15]:
pred_path='prediction'

In [16]:
checkpoint=torch.load('best_checkpoint3.model')
model=ConvNet(num_classes=10)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrelu1): LeakyReLU(negative_slope=0.01)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (lrelu2): LeakyReLU(negative_slope=0.01)
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrelu3): LeakyReLU(negative_slope=0.01)
  (fc): Linear(in_features=180000, out_features=10, bias=True)
)

In [17]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [18]:
images_path=glob.glob(pred_path+'/*.JPEG')

In [19]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [20]:
pred_dict

{'prediction\\1.JPEG': 'keyboard',
 'prediction\\10.JPEG': 'tower',
 'prediction\\12.JPEG': 'orange',
 'prediction\\13.JPEG': 'bucket',
 'prediction\\14.JPEG': 'banana',
 'prediction\\15.JPEG': 'tower',
 'prediction\\16.JPEG': 'penguin',
 'prediction\\17.JPEG': 'orange',
 'prediction\\18.JPEG': 'tea',
 'prediction\\19.JPEG': 'orange',
 'prediction\\2.JPEG': 'orange',
 'prediction\\20.JPEG': 'keyboard',
 'prediction\\21.JPEG': 'keyboard',
 'prediction\\22.JPEG': 'schoolbus',
 'prediction\\23.JPEG': 'pizza',
 'prediction\\24.JPEG': 'pizza',
 'prediction\\25.JPEG': 'tower',
 'prediction\\26.JPEG': 'keyboard',
 'prediction\\27.JPEG': 'orange',
 'prediction\\28.JPEG': 'pizza',
 'prediction\\29.JPEG': 'schoolbus',
 'prediction\\3.JPEG': 'pizza',
 'prediction\\30.JPEG': 'banana',
 'prediction\\31.JPEG': 'banana',
 'prediction\\32.JPEG': 'tea',
 'prediction\\33.JPEG': 'pizza',
 'prediction\\34.JPEG': 'orange',
 'prediction\\35.JPEG': 'keyboard',
 'prediction\\36.JPEG': 'elephant',
 'prediction